<a href="https://colab.research.google.com/github/MengOonLee/WebScrapy/blob/master/CountyHoliday/CountryHoliday01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install --no-cache-dir -qU scrapy

In [ ]:
%%bash
rm -rf Data

In [ ]:
import os
import pathlib
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.loader import ItemLoader
from itemloaders.processors import MapCompose, TakeFirst
from w3lib.html import remove_tags
import time

input_dir = os.path.join(".", "Data/Input")
os.makedirs(input_dir, exist_ok=True)

output_dir = os.path.join(".", "Data/Output")
os.makedirs(output_dir, exist_ok=True)

class HolidayItem(scrapy.Item):
    country = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )

class HolidaySpider(scrapy.Spider):
    name = "Holiday"

    def start_requests(self):
        urls = [
            "https://www.timeanddate.com/holidays/"
            # f"file://{os.getcwd()}/{input_dir}/main.html"
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_main)

    def parse_main(self, response):
        for s in response.css("ul.category-list__list > li"):
            time.sleep(5)
            self.country = s.css("::text").get()
            url = s.xpath("./a/@href").get()
            if url is not None:
                yield response.follow(url=url, callback=self.parse_country)

    def parse_country(self, response):
        print(self.country)
        # l = ItemLoader(item=HolidayItem(), selector=response)
        # l.add_value("country", self.country)
        # yield l.load_item()
        # country_dir = os.path.join(input_dir, self.country)
        # filename = os.path.join(country_dir, "main.html")
        # filename = os.path.join(input_dir, "main.html")
        # pathlib.Path(filename).write_bytes(response.body)
        # self.log(f"Saved file: {filename}")

process = CrawlerProcess()
process.crawl(HolidaySpider)
process.start()

In [ ]:
import os
import time
import pathlib
import scrapy
from scrapy.crawler import CrawlerProcess

class HtmlSpider(scrapy.Spider):
    name = "html"

    def start_requests(self):
        urls = ["https://www.timeanddate.com/holidays"]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_front)

    def parse_front(self, response):
        self.input_dir = os.path.join(os.getcwd(), 'Data/Input')
        os.makedirs(self.input_dir, exist_ok=True)

        page = response.url.split("/")[-2]
        filename = f"{self.input_dir}/{page}.html"
        pathlib.Path(filename).write_bytes(response.body)
        self.log(f"Saved file {filename}")

        for country in response.css('ul.category-list__list > li'):
            time.sleep(3)
            country_name = country.xpath(".//text()").get()
            self.country_dir = os.path.join(self.input_dir, country_name)
            os.makedirs(self.country_dir, exist_ok=True)

            url = country.xpath("./a/@href").get()
            url = url + "2023?hol=1"
            print(country_name, url)
            if url is not None:
                yield response.follow(url=url, callback=self.parse_page)

    def parse_page(self, response):
        filename = f"{self.country_dir}/2023.html"
        pathlib.Path(filename).write_bytes(response.body)
        self.log(f"Saved file {filename}")

process = CrawlerProcess()
process.crawl(HtmlSpider)
process.start()

In [ ]:
import os
import scrapy

input_dir = "./Data/Input"
with open(os.path.join(input_dir, 'front.html'), 'r') as file:
  html = file.read()

sel = scrapy.Selector(text=html)
blocks = sel.css("ul.category-list__list > li")
countries = blocks.xpath(".//text()").getall()
links = blocks.xpath("./a/@href").getall()
links

In [ ]:
import os
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.loader import ItemLoader
from itemloaders.processors import MapCompose, TakeFirst
from w3lib.html import remove_tags

base_dir = os.getcwd()
input_dir = "Data/Input"
output_dir = "Data/Output"

class HolidayItem(scrapy.Item):
    country = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )


class HolidaysSpider(scrapy.Spider):
    name = 'Holidays'

    def start_requests(self):
        urls = [
            f"file://{base_dir}/{input_dir}/front.html"
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_front)

    def parse_front(self, response):
        for country in response.css('ul.category-list__list > li'):
            l = ItemLoader(item=HolidayItem(), selector=country)
            l.add_xpath("country", ".//text()")
            link = country.xpath("./a/@href").get()
            link = link + "/2023?hol=1"
            print(link)
            yield l.load_item()

    # def parse_pages(self, response):
    #     tables = response.css('table#holidays-table')
    #     dates = tables.xpath('.//th[@class="nw"]/text()').getall()
    #     print(dates)

process = CrawlerProcess(
    settings={
        "FEEDS":{f"{base_dir}/{output_dir}/holiday.jl":{"format":"jsonlines"}}
    }
)
process.crawl(HolidaysSpider)
process.start()

In [ ]:
import scrapy
import requests

url = 'https://www.timeanddate.com/holidays/afghanistan/2023?hol=1'
html = requests.get(url).content

sel = scrapy.Selector(text=html)
tables = sel.css('table#holidays-table')
tables
# dates = sel.css('th.nw::text').extract()
# holidays = sel.css('td > a::text').extract()
